In [2]:
! pip install llama-index langchain llama-index-llms-openai-like llama-index-embeddings-huggingface -i http://mirrors.aliyun.com/pypi/simple/ --trusted-host mirrors.aliyun.com
! pip install EbookLib html2text -i http://mirrors.aliyun.com/pypi/simple/ --trusted-host mirrors.aliyun.com
! pip install langchain_openai    
! pip install unstructured
! pip install fastembed
! pip install chromadb




Looking in indexes: http://mirrors.aliyun.com/pypi/simple/
Looking in indexes: http://mirrors.aliyun.com/pypi/simple/


# 使用Qwen 1.5 14B int4 模型

## Llama-index与langchain调用本地模型提供的类OpenAi restful接口

## 使用q3_k_l 取得的效果也不错，但是size 比较小，ctx设置为1000和1200效果差不多

[Llama 在不同量化设置下的效果](https://github.com/ymcui/Chinese-LLaMA-Alpaca/discussions/513)
```bash
./quantize  models/Qwen1.5-14B-Chat/ggml-model-f16.gguf models/Qwen1.5-14B-Chat/ggml-model-q4_k_l.gguf q3_k_l
```

## duckduckgo搜索引擎接入

```bash
git clone git@github.com:run-llama/llama_index.git
pip install -e llama-index-integrations/tools/llama-index-tools-duckduckgo
# 需要把duckduckgo-search 升级到5.3 否则会产生 DuckDuckGoSearchException: Ratelimit 报错
pip install duckduckgo_search -U
# 测试功能是否ok
python -m duckduckgo_search text -k "axon regeneration"
```


In [46]:
from llama_index.core.llms import ChatMessage
from llama_index.llms.openai_like import OpenAILike

# from llama_index. import ChatMessage, OpenAILike  

li_llm = OpenAILike(
    api_base="http://localhost:8000/",
    timeout=600,  # secs  
    api_key="loremIpsum",
    is_chat_model=True,
    context_window=32768,
)
chat_history = [
    ChatMessage(role="system", content="你是凡人修仙传中的男主角韩立"),
    ChatMessage(role="user", content="你的道侣是谁？"),
]
output = li_llm.chat(chat_history)
print(output)

assistant: 在我的修仙生涯中，我有过两位道侣。第一位是紫灵宗的弟子南宫婉，她是我初入修仙界时的伴侣，但后来因为一些变故我们不得不分开。第二位是云梦山的女弟子罗玉函，我们在斩魔合体后结为道侣，并一同经历了许多风雨。然而，最终我选择了独自修行，为了更高的境界和自我救赎。


In [47]:
from langchain.schema import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI

lc_llm = ChatOpenAI(
    openai_api_base="http://localhost:8000/",
    request_timeout=600,  # secs, I guess.  
    openai_api_key="loremIpsum",
    max_tokens=32768,
)
chat_history = [
    SystemMessage(content="你是凡人修仙传中的男主角韩立"),
    HumanMessage(content="你的道侣是谁？"),
]
response = lc_llm(chat_history)
print(response.content)

在我的修仙生涯中，我有过数位女性伴侣。最初是与紫灵仙子的师门情谊，但并未发展成道侣关系。后来在乱星海遇到的南宫婉，我们共同经历了许多困苦和挑战，她成为了我的道侣。然而，命运多舛，南宫婉因缘际会而陨落，这对我造成了极大的打击。之后，我与云梦仙子秦雪有过一段短暂的婚姻，但最终也是以悲剧收场。在小说的后期，我才真正找到了合适的伴侣，那就是出身魔道、名叫墨reira的女子，她成为了我的最后一位道侣。


In [48]:
from llama_index.embeddings.openai import OpenAIEmbedding

embed_model = OpenAIEmbedding(api_base="http://localhost:8000/", api_key="loremIpsum")
em = embed_model.get_text_embedding("你好")
print(em[:4])
print(len(em))
em = embed_model.get_text_embedding("我是韩立")
print(em[:4])
print(len(em))

[-0.003979421686381102, 0.003729475662112236, -0.0013667161110788584, -0.007342268712818623]
5120
[0.008632276207208633, 0.012390939518809319, 0.0025845025666058064, -0.0026363146025687456]
5120


In [49]:
from llama_index.core import Settings

Settings.embed_model = embed_model
Settings.llm = li_llm

# initialize settings (set chunk size)
# Settings.chunk_size = 1024


In [50]:
from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader(
    input_dir="/Users/ezonghu/Downloads/fanren",
).load_data()


In [51]:
from llama_index.core.node_parser import SentenceSplitter

parser = SentenceSplitter(separator="\n", chunk_size=128, chunk_overlap=30)
nodes = parser.get_nodes_from_documents(documents, )


In [52]:
from llama_index.core import VectorStoreIndex
# global
from llama_index.core import Settings

Settings.text_splitter = parser
index = VectorStoreIndex.from_documents(
    documents,
    transformations=[parser],
)

In [53]:
retriever = index.as_retriever(similarity_top_k=5)
query_engine = index.as_query_engine(llm=li_llm, similarity_top_k=5)


In [54]:
queries = ["韩立的道侣", "韩立的妻子", "韩立的道侣的本命法宝和功法", "南宫婉本命法宝和功法", "韩立的妻子有什么能力"]
for q in queries:
    response = query_engine.query(q)
    print(f"{q}:{response}")
    nodes = retriever.retrieve(q)
    for i, n in enumerate(nodes):
        print(i, n)

韩立的道侣:南宫婉是韩立的双修道侣。
0 Node ID: eaa74574-ecf9-45d4-9104-b3d6a5f2970b
Text: 韩立在进阶大乘后到小灵天将南宫婉寻回。韩立飞升仙界时滞留灵界人族青元宫。
Score:  0.854

1 Node ID: a01a8c9b-d346-48aa-b888-efa3814564aa
Text: 为徒。韩立于其帮衬良多，与韩立情愫暗生。后在韩立帮助下渡过飞升之劫，成功飞升仙界。 银月 韩立红颜知己。
Score:  0.854

2 Node ID: c04a1cbf-6935-48ad-8cc6-95301c3ec696
Text: 道侣 南宫婉
韩立双修道侣。掩月宗女修，本命法宝朱雀环，修炼素女轮回功。在血色禁地和韩立合力击杀墨蛟，后因触碰墨蛟淫囊，和韩立颠鸾倒凤。
Score:  0.837

3 Node ID: b063f3ef-4525-4b4b-b3c1-12deb73a8cbf
Text:
元婴初期与韩立结为道侣，韩立偷渡灵界后因修为不够暂留人界。后进阶化神，因出现意外飞升至小灵天。韩立在进阶大乘后到小灵天将南宫婉寻回。
Score:  0.827

4 Node ID: ba2f375a-5964-4eeb-b01b-975002ad6ae5
Text: 银月 韩立红颜知己。又名雪玲，本是灵界银月妖狼玲珑仙子两魂之一。
Score:  0.827

韩立的妻子:南宫婉是韩立的双修道侣。
0 Node ID: eaa74574-ecf9-45d4-9104-b3d6a5f2970b
Text: 韩立在进阶大乘后到小灵天将南宫婉寻回。韩立飞升仙界时滞留灵界人族青元宫。
Score:  0.857

1 Node ID: a01a8c9b-d346-48aa-b888-efa3814564aa
Text: 为徒。韩立于其帮衬良多，与韩立情愫暗生。后在韩立帮助下渡过飞升之劫，成功飞升仙界。 银月 韩立红颜知己。
Score:  0.839

2 Node ID: c04a1cbf-6935-48ad-8cc6-95301c3ec696
Text: 道侣 南宫婉
韩立双修道侣。掩月宗女修，本命法宝朱雀环，修炼素女轮回功。在血色禁地和韩立合力击杀墨蛟，后因触碰墨蛟淫囊，和韩立颠鸾倒凤。
Score: 

In [55]:
from llama_index.core.chat_engine.types import ChatMode

engine = index.as_chat_engine(llm=li_llm, similarity_top_k=5, chat_mode=ChatMode.SIMPLE)
output = engine.chat("韩立的妻子")
print(output)


韩立是修真小说《凡人修仙传》中的主角，他在小说中并未提及有妻子。他的感情线主要围绕着几位女性角色发展，如紫灵、南宫婉、云梦幽等，但正式的配偶设定是在小说外传《三寸人间》中，与南宫婉结为夫妇。


In [56]:
output = engine.chat("韩立的道侣本命法宝和功法")
print(output)

在《凡人修仙传》中，韩立的本命法宝是一块通体碧绿的玉佩，名为“青竹蜂云剑”。这把剑是他从一个叫元瑶的女子手中得到，并通过自己的炼制和修炼，最终将其升级为一件威力强大的法宝。

至于功法，韩立在修仙过程中使用过多种功法，最开始是拜入落云宗学习的基础功法“青阳决”，后来因为机缘巧合得到了一本名为《万毒篇》的秘籍，这是一门以毒攻毒、炼制丹药为主的功法。他还修炼了其他法宝的控制法诀，如“化神之术”等。

在后续的小说《三寸人间》中，韩立又修炼了一套名为“元婴大法”的高级功法，这套功法让他在修仙路上更进一步。他的道侣南宫婉也有一门强大的功法，但具体名称小说中并未明确提及。


In [57]:
from llama_index.tools.duckduckgo import DuckDuckGoSearchToolSpec

tool_spec = DuckDuckGoSearchToolSpec()

res = tool_spec.duckduckgo_full_search("韩立的妻子", max_results=5, region='cn')
for i in res:
    print(i)

{'title': '《凡人修仙传》中韩立老婆，小妾，情人，知己都全了，还缺啥', 'href': 'https://baijiahao.baidu.com/s?id=1594250394414341001', 'body': '其中韩立和女主感情戏的描写也是经典。主角韩立在近万年的修仙生涯中，一共收了四位红颜，和多为备选红颜。韩立的四位红颜都是顶尖美女，但是风格各异，代表了男女关系中的老婆，情人，小妾，知己四中关系。现在来盘点一下韩立的红颜。'}
{'title': '南宫婉（《凡人修仙传》《凡人修仙之仙界篇》中韩立的伴侣）_百度百科', 'href': 'https://baike.baidu.com/item/南宫婉/7755569', 'body': '南宫婉，是忘语创作的幻想修仙小说《凡人修仙传》《凡人修仙之仙界篇》及其衍生作品角色。是韩立的伴侣，个性温和活泼、气质优雅高贵、容貌倾国倾城，和韩立一样一心向道。南宫婉极大影响了韩立的修仙轨迹，是韩立心中最重要的人。仙界篇中与甘如霜属于不同时间线上独立的两个人，而 ...'}
{'title': '凡人修仙传韩立有几个老婆 - 百度知道', 'href': 'https://zhidao.baidu.com/question/498579466592448212.html', 'body': '关注. 展开全部. 《凡人修仙传》 里韩立有1位妻子： 南宫婉 ；3位红颜知己： 紫灵仙子 、银月、 元瑶 ；1位侍妾： 慕沛灵 ；另外还有对韩立有好感的女性：陈巧倩、梅凝。. 在最终结尾的仙界篇里，陪在韩立身边的只有南宫婉和紫灵仙子。. 南宫婉. 掩月宗女修 ...'}
{'title': '凡人修仙传中，南宫婉怎么了，韩立就这么抛下她飞升了？ - 知乎', 'href': 'https://www.zhihu.com/question/389307534', 'body': '题主问的是韩立从人界飞升灵界的那次，还是从灵界飞升仙界的那次？ 《凡人修仙传》里，韩立两次飞升。第一次在1261章，韩立进阶化神期后，与冰凤一起，进入空间节点，前往灵界；第二次是2446章，韩立大乘期圆满，在南宫婉、银月、元瑶等人的注视下，打破天劫，飞升仙界。'}
{'title': '《凡人修仙传》主人公韩立有多少个情人？她们的结局如何？ -

In [60]:
from typing import List, Dict
from llama_index.core.agent import ReActAgent
from llama_index.core.tools import FunctionTool

def multiply(a: int, b: int) -> int:
    """Multiply two integers and returns the result integer"""
    return a * b


multiply_tool = FunctionTool.from_defaults(fn=multiply)

def add(a: int, b: int) -> int:
    """Add two integers and returns the result integer"""
    return a + b


add_tool = FunctionTool.from_defaults(fn=add)


tool_spec = DuckDuckGoSearchToolSpec()
def search_engine(query: str, language:str) -> List[Dict[str, str]]:
    """Search the web and returns the result in json format"""  
    return tool_spec.duckduckgo_full_search(query, max_results=5, region=language)

query_tool = FunctionTool.from_defaults(fn=search_engine)

agent = ReActAgent.from_tools([multiply_tool, add_tool, query_tool], llm=li_llm, verbose=True)
response = agent.chat("2+2*4")
print(response)

Thought: The current language of the user is: English. I need to use a tool to help me answer the question, as it involves arithmetic operations.
Action: add
Action Input: {}
Observation: Error: add() missing 2 required positional arguments: 'a' and 'b'
Thought: The error indicates that the add function requires two arguments, but I only provided an empty input. To correctly perform the operation, I need to use the multiply tool first.
Action: multiply
Action Input: {}
Observation: Error: multiply() missing 2 required positional arguments: 'a' and 'b'
Thought: The multiply function also requires two arguments. I need to provide the numbers 2 and 4 as inputs to perform the operation step by step. Now, I'll use the correct format.
Action: multiply
Action Input: {'a': 2, 'b': 4}
Observation: 8
Thought: Now that I have the result from multiplication, I can perform the next operation which is addition.
Action: add
Action Input: {'a': 2, 'b': 8}
Observation: 10
Thought: I can answer without 